In [1]:
!pip install transformers
!pip install wget

     |████████████████████████████████| 2.3MB 8.6MB/s 
     |████████████████████████████████| 901kB 49.9MB/s 
     |████████████████████████████████| 3.3MB 44.1MB/s 
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=e8071a2e497a89da108229b859bfd31d390d54e6a92c7f236aaa1cd373280e15
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [12]:
# Datenvorbereitung
import json
import wget

# download and open json file
url = 'https://raw.githubusercontent.com/Johannes96/BERT_fineTune/master/data/final_train_ae_CLS.json'
dataset1 = wget.download(url)

with open(dataset1) as json_file:
    train_ae = json.load(json_file)

url =  'https://raw.githubusercontent.com/Johannes96/BERT_fineTune/master/data/final_dev_ae_CLS.json'
dataset2 = wget.download(url)

with open(dataset2) as json_file:
    val_ae = json.load(json_file)

# Extrahiere Sätze und Labels
train_texts = []
train_tags = []
for i in train_ae:
    train_tags.append(train_ae[i]['label'])
    train_texts.append(train_ae[i]['sentence'])

val_texts = []
val_tags = []
for i in val_ae:
    val_tags.append(val_ae[i]['label'])
    val_texts.append(val_ae[i]['sentence'])

# Vereinige Trainings- und Testdaten
texts = train_texts
tags = train_tags
tags.extend(val_tags)
texts.extend(val_texts)

In [13]:
# Bereinige Datensatz von '\x85'
for i, sent in enumerate(texts):
  for i2, word in enumerate(texts[i]):
    if word == '\x85':
      print('sent:', i, 'word:',i2)
      del texts[i][i2]
      del tags[i][i2]

sent: 3600 word: 5
sent: 3655 word: 5
sent: 3924 word: 5
sent: 6585 word: 5


In [14]:
# Teile in Trainigns- und Testdaten auf
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

In [17]:
# unique_tags = set(tag for doc in train_tags for tag in doc)
unique_tags = {'O', 'B', 'I'}
# tag2id = {tag: id for id, tag in enumerate(unique_tags)}
tag2id = {'O': 0, 'B': 1, 'I': 2}
# id2tag = {id: tag for tag, id in tag2id.items()}
id2tag = {0: 'O', 1: 'B', 2: 'I'}
print(tag2id)
print(id2tag)
print(unique_tags)

{'O': 0, 'B': 1, 'I': 2}
{0: 'O', 1: 'B', 2: 'I'}
{'B', 'O', 'I'}


In [18]:
# Tokenization
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [19]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for i, (doc_labels, doc_offset) in enumerate(zip(labels, encodings.offset_mapping)):
        # create an empty array of -100
        # print(i)
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)
        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())
    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [20]:
import torch

class AEDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = AEDataset(train_encodings, train_labels)
val_dataset = AEDataset(val_encodings, val_labels)

In [21]:
# Spezifiziere Anzahl der labels
from transformers import BertForTokenClassification
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(unique_tags))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [22]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Step,Training Loss
10,1.283600
20,1.196500
30,1.021900
40,0.780800
50,0.513800
60,0.317300
70,0.307900
80,0.238200
90,0.244000
100,0.200100


TrainOutput(global_step=1152, training_loss=0.1312014089198783, metrics={'train_runtime': 495.8638, 'train_samples_per_second': 2.323, 'total_flos': 190823036651280.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2368700416, 'init_mem_gpu_alloc_delta': 431416832, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 105902080, 'train_mem_gpu_alloc_delta': 1333511168, 'train_mem_cpu_peaked_delta': 69632, 'train_mem_gpu_peaked_delta': 1701886976})

In [23]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_loss': 0.1318761259317398,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 235922944,
 'eval_runtime': 10.2309,
 'eval_samples_per_second': 149.938}

In [24]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-016dc04f-637b-1e19-fcb4-de4d4d376a2d)


In [39]:
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

class AEDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

val_encodings.pop("offset_mapping")
val_dataset = AEDataset(val_encodings)

In [52]:
print(val_dataset.encodings)

{'input_ids': [[101, 7845, 118, 1253, 170, 2113, 3945, 1183, 6103, 1195, 1125, 170, 109, 1512, 5309, 10890, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1284, 2802, 170, 9670, 1104, 1472, 12647, 26883, 26542, 1105, 1152, 1127, 1155, 1541, 1363, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 146, 1225, 183, 112, 189, 1541, 1138, 170, 2463, 1114, 1103, 1353, 7072, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [40]:
predictions = trainer.predict(val_dataset)

In [97]:
pred_labels = []
val_labels2 = []
for i, pred in enumerate(predictions.predictions):
  max_len = len(val_texts[i])
  pred_labels.append(pred[0:max_len])
  val_labels2.append(val_labels[i][0:max_len])
print(pred_labels[0])

[[ 5.118797   -1.1109351  -3.802988  ]
 [-0.4430694   4.2703466  -3.450832  ]
 [ 6.957081   -3.8029308  -3.711654  ]
 [ 6.843724   -2.7379894  -4.5674963 ]
 [ 6.607854   -2.8851638  -4.4550343 ]
 [ 6.212617   -2.2230434  -4.4794955 ]
 [ 5.0176716  -1.6676799  -4.2437716 ]
 [ 6.0016413  -3.006174   -3.9353461 ]
 [ 6.9359236  -3.338157   -4.3822107 ]
 [ 7.068423   -3.6742134  -4.0559335 ]
 [ 6.897196   -3.404871   -4.2557597 ]
 [ 6.839278   -3.4998078  -4.1729665 ]
 [ 5.2805758  -1.1011581  -4.69445   ]
 [ 5.1738024  -3.9343305  -1.6367483 ]
 [ 3.267172    0.30660275 -3.0755665 ]]


In [94]:
print(val_texts[0][0:15])
print(len(val_texts[0]))
print(val_labels2[0])
print(pred_labels[0])

['finance', '-', 'still', 'a', 'bit', 'pricey', 'considering', 'we', 'had', 'a', '$', '25', 'gift', 'certificate', '!']
15
[-100, 1, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0]
[[ 5.118797   -1.1109351  -3.802988  ]
 [-0.4430694   4.2703466  -3.450832  ]
 [ 6.957081   -3.8029308  -3.711654  ]
 [ 6.843724   -2.7379894  -4.5674963 ]
 [ 6.607854   -2.8851638  -4.4550343 ]
 [ 6.212617   -2.2230434  -4.4794955 ]
 [ 5.0176716  -1.6676799  -4.2437716 ]
 [ 6.0016413  -3.006174   -3.9353461 ]
 [ 6.9359236  -3.338157   -4.3822107 ]
 [ 7.068423   -3.6742134  -4.0559335 ]
 [ 6.897196   -3.404871   -4.2557597 ]
 [ 6.839278   -3.4998078  -4.1729665 ]
 [ 5.2805758  -1.1011581  -4.69445   ]
 [ 5.1738024  -3.9343305  -1.6367483 ]
 [ 3.267172    0.30660275 -3.0755665 ]]


In [98]:
# Erstelle Liste mit predicted values
pred_labels2 = []
for i, sent in enumerate(pred_labels):
  pred_labels2.append([])
  for ix, word in enumerate(sent):
    idx = max(range(len(pred_labels[i][ix])), key=pred_labels[i][ix].__getitem__)
    pred_labels2[i].append(idx)
print(pred_labels2)

[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 

In [99]:
print(pred_labels2[0])
print(val_labels2[0])
# Entferne alle Werte mit 0 oder -100
for i, preds in enumerate(val_labels2):
  for idx, word in enumerate(val_labels2[i]):
    if word == -100 or word == 0:
      # print(idx)
      pred_labels2[i].pop(idx)
      val_labels2[i].pop(idx)
print(pred_labels2[0])
print(val_labels2[0])

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 1, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, -100, 0, 0, 0]
